In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Install + Load Tokenizer

In [2]:
!pip -q install -U transformers datasets accelerate

from transformers import AutoTokenizer
from datasets import Dataset

MODEL_ID = "Qwen/Qwen2.5-1.5B-Instruct"
tok = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
tok.padding_side = "right"
tok.truncation_side = "right"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 10.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Load My CSV

In [ ]:
import pandas as pd
import json


CSV_PATH = "/content/drive/MyDrive/novel_paragraphs_6kself_plus_transfer.csv"  # change if needed
df = pd.read_csv(CSV_PATH)

# quick sanity check
# print(df.mode.value_counts())
# print(df.source_style.value_counts().head())
# print(df.target_style.value_counts().head())

# convert to HF Dataset
rows = df.to_dict(orient="records")
ds = Dataset.from_list(rows)
ds

Dataset({
    features: ['mode', 'source_style', 'target_style', 'input', 'output', 'raw_content'],
    num_rows: 9575
})

Build chat messages per row

For transfer: “{source_style} → {target_style}” rewrite with names/events preserved, answer is output.

For self: identity rewrite in the same style, answer is output (or raw_content if output empty).


In [5]:
def row_to_messages(row):
    src = (row.get("raw_content") or "").strip()
    tgt = (row.get("output") or "").strip()
    src_style = (row.get("source_style") or "").strip()
    tgt_style = (row.get("target_style") or "").strip()
    mode = (row.get("mode") or "self").strip()

    system = "You are a careful literary rewriting assistant that follows instructions exactly."

    if mode == "transfer" and tgt_style and tgt:
        user = (
            f"Rewrite the paragraph from **{src_style}** into **{tgt_style}** style.\n"
            f"Rules: preserve all character names and main events; change only tone, narration, and imagery.\n\n"
            f"Paragraph:\n{src}"
        )
        assistant = tgt
    else:
        # self mode: same-style rewrite to teach faithful copying
        user = (
            f"Rewrite the paragraph in **{src_style or tgt_style}** style without changing events or names.\n\n"
            f"Paragraph:\n{src}"
        )
        assistant = tgt or src

    return [
        {"role": "system", "content": system},
        {"role": "user", "content": user},
        {"role": "assistant", "content": assistant},
    ]

def add_prompt(example):
    messages = row_to_messages(example)
    prompt = tok.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False  # training, not inference
    )
    example["prompt"] = prompt
    return example

ds_chat = ds.map(add_prompt)
print(ds_chat[0]["prompt"][:500])


Map:   0%|          | 0/9575 [00:00<?, ? examples/s]

<|im_start|>system
You are a careful literary rewriting assistant that follows instructions exactly.<|im_end|>
<|im_start|>user
Rewrite the paragraph in **comedy** style without changing events or names.

Paragraph:
"No, no, no! What yer got? what yer got? Gie me somethin', gie me somethin'. Settle, settle, settle! Gie me any thin' yer got. Settle, settle, settle!" The consequences of twenty years' such traffic as this can more easily be imagined than described. The room was piled from floor to 


In [ ]:
MAX_LEN = 512  # 512–2048 are common for LoRA on a T4

def tokenize_fn(batch):
    enc = tok(
        batch["prompt"],
        padding="max_length",
        truncation=True,
        max_length=MAX_LEN,
        return_tensors=None,
    )
    enc["labels"] = enc["input_ids"].copy()
    return enc

cols_keep = ["input_ids", "attention_mask", "labels"]
ds_tok = ds_chat.map(tokenize_fn, batched=True, remove_columns=list(ds_chat.column_names))
ds_tok = ds_tok.with_format("torch", columns=cols_keep)
ds_tok


Map:   0%|          | 0/9575 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 9575
})

In [ ]:
save_dir = "/content/drive/MyDrive/qwen2_sft_ds_tok"
ds_tok.save_to_disk(save_dir)          



Saving the dataset (0/1 shards):   0%|          | 0/9575 [00:00<?, ? examples/s]